In [ ]:
import pymongo
import pandas as pd

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["animal_rescue"]
animals_collection = db["animals"]

# Read Excel file into Pandas DataFrame
df = pd.read_excel('animal_data.xlsx') 

# Convert DataFrame to dictionary
data = df.to_dict('records') 

# Insert into MongoDB collection
animals_collection.insert_many(data)